In [1]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = '3718b3ee547542b48915cd97527ae072'
CLIENT_SECRET = '873b12eaa6934c8a97a10e9a3e59853f'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [3]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


pip install spotipy

In [4]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id,name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)
    
    return df

In [5]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

In [6]:
# Display the DataFrame
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Not Like Us,Kendrick Lamar,Not Like Us,5JjnoGJyOxfSZUZtk2rRwZ,6AI3ezQ4o3HUoP6Dhudph3,97,2024-05-04,274192,True,https://open.spotify.com/track/6AI3ezQ4o3HUoP6...,...,0.472,1,-7.001,1,0.0776,0.01070,0.000000,0.1410,0.214,101.061
1,Houdini,Eminem,Houdini,6Xuu2z00jxRPZei4IJ9neK,2HYFX63wP3otVIvopRS99Z,92,2024-05-31,227239,True,https://open.spotify.com/track/2HYFX63wP3otVIv...,...,0.887,9,-2.760,0,0.0683,0.02920,0.000002,0.0582,0.889,127.003
2,I Don't Wanna Wait,"David Guetta, OneRepublic",I Don't Wanna Wait,0wCLHkBRKcndhMQQpeo8Ji,331l3xABO0HMr1Kkyh2LZq,91,2024-04-05,149668,False,https://open.spotify.com/track/331l3xABO0HMr1K...,...,0.714,1,-4.617,0,0.0309,0.03750,0.000000,0.2320,0.554,129.976
3,BAND4BAND (feat. Lil Baby),"Central Cee, Lil Baby",BAND4BAND (feat. Lil Baby),4AzPr5SUpNF553eC1d3aRy,7iabz12vAuVQYyekFIWJxD,91,2024-05-23,140733,True,https://open.spotify.com/track/7iabz12vAuVQYye...,...,0.764,11,-5.241,1,0.2040,0.35900,0.000000,0.1190,0.886,140.113
4,Pedro,"Jaxomy, Agatino Romero, Raffaella Carrà",Pedro,5y6RXjI5VPR0RyInghTbf1,48lxT5qJF0yYyf2z4wB4xW,88,2024-03-29,144846,False,https://open.spotify.com/track/48lxT5qJF0yYyf2...,...,0.936,9,-6.294,1,0.3010,0.02290,0.000001,0.3110,0.844,151.019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Edge of Seventeen,Wuki,Edge of Seventeen,6oeEtPWpOfed7plMHI5qef,5IFSvSEeCAvvBEQ85poqKB,71,2022-10-21,164043,False,https://open.spotify.com/track/5IFSvSEeCAvvBEQ...,...,0.909,5,-5.999,0,0.0476,0.03240,0.527000,0.0800,0.351,127.997
96,Rhyme Dust,"MK, Dom Dolla",Rhyme Dust,1vSO4N7cwo4pBywCSGF5BR,59QDyqLww2pxyg9ijOPO7f,70,2023-02-24,181365,False,https://open.spotify.com/track/59QDyqLww2pxyg9...,...,0.561,3,-8.675,1,0.1760,0.06590,0.027100,0.1520,0.721,128.034
97,RIZZ,AYYBO,RIZZ,6uH0vo25qVqNveP02Pr4Uc,3DqOJ2BoddzZpNdty3okJH,70,2023-11-15,200787,True,https://open.spotify.com/track/3DqOJ2BoddzZpNd...,...,0.839,7,-5.733,1,0.1530,0.07360,0.426000,0.3200,0.729,127.044
98,i like the way you kiss me - southstar remix,"Artemas, southstar",i like the way you kiss me (remixes),7jYsvUIbj8OktGfdnCU4LB,7skteOnDbwZ3ZvjSe99xv2,69,2024-04-05,137987,False,https://open.spotify.com/track/7skteOnDbwZ3Zvj...,...,0.876,11,-5.973,1,0.0449,0.00352,0.462000,0.2350,0.200,154.989


In [7]:
music_df.isnull().sum()

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
data = music_df

In [11]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span 
    #(e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [12]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability',
                           'Energy', 
                           'Key', 
                           'Loudness',
                           'Mode',
                           'Speechiness',
                           'Acousticness',
                           'Instrumentalness',
                           'Liveness', 
                           'Valence',
                           'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [13]:
music_features_scaled

array([[0.89878543, 0.14383562, 0.09090909, ..., 0.14638958, 0.18800214,
        0.33487784],
       [0.9757085 , 0.85445205, 0.81818182, ..., 0.0374852 , 0.91108731,
        0.55342881],
       [0.45951417, 0.55821918, 0.09090909, ..., 0.26607918, 0.55222282,
        0.57847515],
       ...,
       [0.81174089, 0.77226027, 0.63636364, ..., 0.38182296, 0.73968934,
        0.55377422],
       [0.40890688, 0.83561644, 1.        , ..., 0.27002499, 0.17300482,
        0.78919966],
       [0.73684211, 0.8510274 , 0.09090909, ..., 0.06418519, 0.5286556 ,
        0.52847515]])

In [15]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]],
                                          music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name',
                                                                         'Artists',
                                                                         'Album Name',
                                                                         'Release Date',
                                                                         'Popularity']]

    return content_based_recommendations

In [16]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name,
                                                      num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name,
                                    'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name,
                                                                                              'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations.append({
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }, ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', 
                                                                ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [17]:
input_song_name = "I'm Good (Blue)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
recommendations

Hybrid recommended songs for 'I'm Good (Blue)':


,Track Name,Artists,Album Name,Release Date,Popularity
4,It's Not Right But It's Okay,Mr. Belt & Wezol,It's Not Right But It's Okay,2024-02-23,83.0
3,Call It Love,"Felix Jaehn, Ray Dalton",Call It Love,2022-09-16,78.0
0,In My Bones,"Lost Frequencies, David Kushner",In My Bones,2024-04-19,75.0
1,REACT,"Switch Disco, Ella Henderson, Robert Miles",REACT,2023-01-13,74.0
2,Where You Are,"John Summit, HAYLA",Where You Are,2023-03-03,72.0


In [18]:
content_based_recommendations(input_song_name, num_recommendations=5)

,Track Name,Artists,Album Name,Release Date,Popularity
70,In My Bones,"Lost Frequencies, David Kushner",In My Bones,2024-04-19,75
71,REACT,"Switch Disco, Ella Henderson, Robert Miles",REACT,2023-01-13,74
90,Where You Are,"John Summit, HAYLA",Where You Are,2023-03-03,72
46,Call It Love,"Felix Jaehn, Ray Dalton",Call It Love,2022-09-16,78
24,It's Not Right But It's Okay,Mr. Belt & Wezol,It's Not Right But It's Okay,2024-02-23,83
